In [1]:
import lightgbm as lgb

In [ ]:
train_data = lgb.Dataset("drive/MyDrive/f_select_train.bin", free_raw_data=False)
train_data.construct()
val_data = lgb.Dataset("drive/MyDrive/f_select_val.bin", reference=train_data, free_raw_data=False)
val_data.construct()
test_data = lgb.Dataset("drive/MyDrive/f_select_test.bin", reference=train_data, free_raw_data=False)
test_data.construct()

In [ ]:
import optuna
from sklearn.metrics import mean_squared_error
from optuna.integration import LightGBMPruningCallback

In [ ]:
def objective(trial, X, y):
    param_grid = {
        "n_estimators" : trial.suggest_categorical("n_estimators", [5000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 500, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 21),
        #"min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100)
        "max_bin":trial.suggest_int("max_bin", 200, 300, step=10),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=10),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=10),
        "main_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.9, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq",[1]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.9, step=0.1),
    }
    model = lgb.LGBMRegressor(objective='regression', **param_grid)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='rmse', early_stopping_rounds=100, callbacks=[LightGBMPruningCallback(trial, "rmse")])
    
    y_pred = model.predict(X_val)
    score = mean_squared_error(y_val, y_pred, squared=False)
    
    return score

In [ ]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)

In [ ]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")